In [1]:
import networkx as nx
import numpy as np
import pandas as pd
from node2vec import Node2Vec
from tqdm import tqdm

In [2]:
import gc

train_data = pd.read_csv("train.csv", 
                    usecols=["node1_id", "node2_id", "is_chat"], 
                    dtype={"node1_id": np.int64, "node2_id":np.int64, "is_chat": np.int8})
test_data = pd.read_csv("test.csv", 
                    usecols=["id", "node1_id", "node2_id"], 
                    dtype={"id": np.int64, "node1_id": np.int64, "node2_id":np.int64})

In [3]:
train_data.head()

,node1_id,node2_id,is_chat
0,8446602,6636127,0
1,1430102,7433949,0
2,2803017,8372333,0
3,4529348,894645,0
4,5096572,4211638,0


In [4]:
train_data.shape

(70661802, 3)

In [5]:
a = list(np.unique(list(train_data['node1_id'].unique()) + list(train_data['node2_id'].unique())))

x_ones = train_data[train_data['is_chat']==1]
x_ones.index = range(0,len(x_ones))

In [6]:
len(a), len(x_ones)

(7997030, 2251784)

In [8]:
max(a)

8874626

In [8]:
#x_ls1=[]
#for i in range(0,len(x)) :
#    x_ls1.append((x['Item1'].iloc[i],x['Item2'].iloc[i],x['Prob'].iloc[i]))


G = nx.Graph()
G.add_nodes_from(a)


x_ls1=[]
for i in range(0,len(x_ones)) :
    x_ls1.append((x_ones['node1_id'].iloc[i], x_ones['node2_id'].iloc[i]))

In [ ]:
# add edges from txt 
G.add_edges_from(x_ls1)

# Generate walks
node2vec = Node2Vec(G, dimensions=16, walk_length=16, num_walks=2, workers=4)

# Learn embeddings 
model = node2vec.fit(window=10, min_count=1)

#a1 = list(model.wv.get_vector('node1'))

Computing transition probabilities: 100%|██████████| 7997030/7997030 [02:34<00:00, 51862.54it/s]
/home/ratulghosh/anaconda3/lib/python3.6/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
Exception in thread QueueManagerThread:
Traceback (most recent call last):
  File "/home/ratulghosh/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/ratulghosh/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ratulghosh/anaconda3/lib/python3.6/site-packages/joblib/externals/loky/process_executor.py", line 708, in _queue_management_worker
    executor._adjust_process_count()
  File "/home/ratulghosh/anaconda3/lib/python3.6/site-packages/joblib/externals/loky/process_executor.py", line 1007, 

In [ ]:
model.wv.save_word2vec_format('graph_embedding_16')

In [9]:

embeddings_index = {}
f = open('graph_embedding')
for line in f:
    values = line.split(' ')
    if len(values) != 2:
        word = values[0] ## The first entry is the word
        coefs = np.asarray(values[1:], dtype='float16') ## These are the vecotrs representing the embedding for the word
        embeddings_index[word] = coefs
f.close()

print('Node2Vec data loaded')

Node2Vec data loaded


In [14]:
embedding_matrix = np.zeros((max(a)+1)*32).reshape((max(a)+1, 32))
for i in tqdm(a):
    embedding_matrix[i] = embeddings_index[str(i)]

100%|██████████| 7997030/7997030 [00:25<00:00, 317930.74it/s]


In [16]:
node_features = pd.read_csv('user_features.csv')

In [17]:
node_features.head()

,node_id,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13
0,2,14,14,14,12,12,12,7,7,7,0,0,0,15
1,3,31,9,7,31,16,12,31,15,12,31,15,12,8
2,4,0,0,0,0,0,0,0,0,0,0,0,0,7
3,5,31,4,1,31,7,1,31,9,1,31,9,0,15
4,6,31,27,20,31,24,14,31,20,10,31,20,5,7


In [18]:
col_lists = ['f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13']
node_features_ = node_features[col_lists].values

In [19]:
feature_index = {}
c=0
for i in tqdm(node_features.node_id.values):
    feature_index[str(i)] = node_features_[c]
    c=c+1

100%|██████████| 8264276/8264276 [00:20<00:00, 406407.62it/s]


In [20]:
feature_matrix = np.zeros((max(a)+1)*13).reshape((max(a)+1, 13))
for i in tqdm(a):
    feature_matrix[i] = feature_index[str(i)]

100%|██████████| 7997030/7997030 [00:21<00:00, 380676.45it/s]


In [23]:
final_matrix = np.concatenate((embedding_matrix, feature_matrix), axis=1)

In [25]:
del node_features, node_features_, embeddings_index, embedding_matrix, feature_index, feature_matrix
gc.collect()

336

In [27]:
train_data.shape

(70661802, 3)

In [35]:
train_data.node1_id.values[10000000]

3858788

In [41]:
node1 = train_data.node1_id.values[0:10000000]
node2 = train_data.node2_id.values[0:10000000]

data = np.zeros(10000000*90).reshape(10000000, 90)
for i in tqdm(range(len(node1))):
    data[i] = np.concatenate((final_matrix[node1[i]], final_matrix[node2[i]]), axis=0)

100%|██████████| 10000000/10000000 [00:24<00:00, 400914.74it/s]


In [45]:
y = train_data.is_chat[0:10000000]

### model

In [48]:
import xgboost as xgb 
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
import os
from sklearn import preprocessing

In [49]:
random_state = 42
np.random.seed(random_state)

lgb_params = {
    "objective" : "binary",
    "metric" : "auc",
    "boosting": 'gbdt',
    "max_depth" : -1,
    "num_leaves" : 13,
    "learning_rate" : 0.01,
    "bagging_freq": 5,
    "bagging_fraction" : 0.4,
    "feature_fraction" : 0.05,
    "min_data_in_leaf": 80,
    "min_sum_heassian_in_leaf": 10,
    "tree_learner": "serial",
    "boost_from_average": "false",
    #"lambda_l1" : 5,
    #"lambda_l2" : 5,
    "bagging_seed" : random_state,
    "verbosity" : 1,
    "seed": random_state
}

In [50]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)

In [59]:
del data
gc.collect()

161

In [60]:
del train_data, test_data
gc.collect()

52

In [61]:
del final_matrix
gc.collect()

0

In [62]:
del node1, node2

In [63]:
gc.collect()

0

In [65]:
del y
gc.collect()

0

In [ ]:
df_train.to_csv('final_data.csv', index=None)

In [56]:
df_train = pd.DataFrame(data)
features = list(df_train.columns)
df_train['target'] = y

In [66]:
for fold, (trn_idx, val_idx) in enumerate(skf.split(df_train, df_train['target'])):
    X_train, y_train = df_train.iloc[trn_idx][features], df_train.iloc[trn_idx]['target']
    X_valid, y_valid = df_train.iloc[val_idx][features], df_train.iloc[val_idx]['target']
    
    N = 5
    p_valid,yp = 0,0
    for i in range(N):
    
        trn_data = lgb.Dataset(X_train, label=y_train)
        val_data = lgb.Dataset(X_valid, label=y_valid)
        evals_result = {}
        lgb_clf = lgb.train(lgb_params,
                        trn_data,
                        100000,
                        valid_sets = [trn_data, val_data],
                        early_stopping_rounds=3000,
                        verbose_eval = 1000,
                        evals_result=evals_result
                       )
        p_valid += lgb_clf.predict(X_valid)
        yp += lgb_clf.predict(X_test)
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = cols
    fold_importance_df["importance"] = lgb_clf.feature_importance()
    fold_importance_df["fold"] = fold + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    oof['predict'][val_idx] = p_valid/N
    val_score = roc_auc_score(y_valid, p_valid)
    val_aucs.append(val_score)
    
    predictions['fold{}'.format(fold+1)] = yp/N

MemoryError: 